In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import preprocessing

Importando os dados:

In [ ]:
adult_train = pd.read_csv("../input/adult-pmr3508/train_data.csv",index_col=['Id'],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")
adult_test = pd.read_csv("../input/adult-pmr3508/test_data.csv",index_col=['Id'],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

In [ ]:
adult_train.shape

In [ ]:
adult_train.head()

Vamos tratar os dados faltantes, primeiro identificando quais parâmetros apresentam dados faltantes.

In [ ]:
Faltantes = adult_train.isnull().sum().sort_values(ascending = False)
Faltantes.head()

In [ ]:
plt.figure(figsize=(17, 7))
adult_train.workclass.value_counts().plot(kind = 'pie')

Vamos atribuir a moda do parâmetro workclass nos dados faltantes dado que grande parte das pessoas trabalham no setor privado.

In [ ]:
plt.figure(figsize=(17, 7))
adult_train['native.country'].value_counts().plot(kind = 'pie')

A maior parte das pessoas desse banco de dados são americanas, é sensato retirar esse parâmetro para fazer o treinamento no fim já que esse parâmetro está muito enviesado para pessoas americanas, se quereremos fazer predição, como não sabemos se serão para americanos ou não é útil retirar esse parametro. No entanto, como a maior parte das pessoas desse banco de dados são americanos é conveninente atribuir a nacionalidade americana aos dados faltantes desse parâmetro.

In [ ]:
plt.figure(figsize=(17, 7))
adult_train.occupation.value_counts().plot(kind = 'pie')

Os dados de occupation são bem distribuídos, no entanto vou substituir os dados faltantes pela moda Prof-specialty mas poderia ser por exemplo Craft-repair por não serem estão distantes em número de ocorrência.

Completando com a moda:

In [ ]:
adult_train["workclass"] = adult_train["workclass"].fillna(adult_train["workclass"].describe().top)
adult_train['native.country'] = adult_train['native.country'].fillna(adult_train['native.country'].describe().top);
adult_train['occupation'] = adult_train['occupation'].fillna(adult_train['occupation'].describe().top);

Verificou-se todos os dados faltantes foram eliminados:

In [ ]:
Faltantes = adult_train.isnull().sum().sort_values(ascending = False)
Faltantes.head()

Realizando o mesmo para os dados de teste:

In [ ]:
adult_test["workclass"] = adult_test["workclass"].fillna(adult_test["workclass"].describe().top)
adult_test['native.country'] = adult_test['native.country'].fillna(adult_test['native.country'].describe().top);
adult_test['occupation'] = adult_test['occupation'].fillna(adult_test['occupation'].describe().top);

In [ ]:
Faltantes = adult_test.isnull().sum().sort_values(ascending = False)
Faltantes.head()

Agora, vou verificar como os parâmetros se relacionam com "income", selecionando os parâmetros mais importantes para o modelo final

In [ ]:
plt.figure(figsize=(15, 7))
adult_train.groupby("income").age.hist()
plt.legend(['<=50k','>50k'])
plt.xlabel('Age')
plt.ylabel('quantity')
plt.title('Age histogram')

A maior parte das pessoas mais bem remuneradas estão na faixa dos 40 anos, é de se esperar dado que nessa idade que se atinge uma certa maturidade profissional. Vale observar que é bem possível que essa concentração nessa idade seja maior ainda do que a apresentada dado que grande parte das pessoas do banco de dados estão na faixa dos vinte anos ainda.

In [ ]:
plt.figure(figsize=(17, 7))
adult_train['education.num'].value_counts().plot(kind = 'pie')

O parametro education será eliminado dado que education.num representa a mesma informação. A maior parte das pessoas estã no nivel de educação 9, 10 e 13. Poucas pessoas tem qualificação superior a essas, mas também poucas pessoas tem abaixo dessas.

In [ ]:
plt.figure(figsize=(17, 7))
adult_train['marital.status'].value_counts().plot(kind = 'pie')

A maior parte dos dados correspondem a pessoas casadas ou não casadas.

In [ ]:
plt.figure(figsize=(17, 7))
adult_train['occupation'].value_counts().plot(kind = 'pie')

In [ ]:
plt.figure(figsize=(17, 7))
adult_train['hours.per.week'].value_counts().plot(kind = 'pie')

A maior parte das pessoas consultadas trabalham 40 horas por semana mas algumas passam disso podendo trabalhar até 60 horas.

In [ ]:
plt.figure(figsize=(17, 7))
adult_train['capital.loss'].value_counts().plot(kind = 'pie')

In [ ]:
plt.figure(figsize=(17, 7))
adult_train['capital.loss'].value_counts().plot(kind = 'pie')

Capital loss e gain não aparentam ser dados importantes pois quase ninguém dos dados apresentam esse tipo de investimento. Porém nos testes feitos retirá-los afeta negativamente o resultado final

In [ ]:
plt.figure(figsize=(20, 7))
adult_train.groupby("income").workclass.hist()
plt.legend(['<=50k','>50k'])
plt.xlabel("Workclass")
plt.ylabel('quantity')
plt.title('Workclass histogram')

In [ ]:
plt.figure(figsize=(20, 7))
adult_train.groupby("income").relationship.hist()
plt.legend(['<=50k','>50k'])
plt.xlabel("Relationship Status")
plt.ylabel('Relationship Status')
plt.title('Relationship histogram')

A maior parte das pessoas bem remuneradas são maridos. No entanto esses dados parecem serem incompletos dado que poucas esposas foram consultadas. Porém pode-se aceitar esse resultado dado que posteriormente veremos que há uma desigualdade de gênero presente na remuneração e portanto em relação a estado civil maridos ganhariam mais do que esposas.

In [ ]:
plt.figure(figsize=(20, 7))
adult_train.groupby("income").race.hist()
plt.legend(['<=50k','>50k'])
plt.xlabel("Race Status")
plt.ylabel('Race Status')
plt.title('Race histogram')

É possivel ver que as pessoas mais bem remuneradas desses dados são caucasianas, no entanto quase todas as pessoas consultadas são caucasianas. Seria interessante que esses dados tivessem mais representatividade de outras raças. Irei considerar esse parâmetro, no entanto ele não parece ser aceitável para países com mais diversidade cultural como o próprio Brasil.

In [ ]:
plt.figure(figsize=(20, 7))
adult_train.groupby("income").sex.value_counts().plot(kind = 'pie')
plt.legend(['<=50k','>50k'])
plt.xlabel("Sex Status")
plt.ylabel('Sex Status')
plt.title('Sex histogram')

Como dito, os dados mostram uma desigualdade de gênero presente na sociedade na questão da remuneração. Das pessoas melhor remuneradas, a grande maioria são homens.

Aqui serão transformadas os parâmetros que não são numéricos em numéricos para poderem ser melhor trabalhados.

In [ ]:
adult_train_num=adult_train.apply(preprocessing.LabelEncoder().fit_transform)
adult_test_num=adult_test.apply(preprocessing.LabelEncoder().fit_transform)

In [ ]:
adult_train_num.head()

Construindo então a matriz de correlação entre parâmetros:

In [ ]:
rel = adult_train_num.drop(['income'],axis = 1)
rel_bi = adult_train_num['income']
rel = pd.concat([rel, rel_bi], axis = 1)
plt.figure(figsize=(15,8))
sns.heatmap(rel.corr(),annot=True)


Dada a matriz de correlação vemos que parâmetros como horas trabalhas por semana, ganho capital, sexo, relacionamento, educação e idade são importantes para o modelo. Parâmetros como fnlwgt pode e vai ser desprezado, education também pois education.num representa a mesma informação e native.country dado que esse parâmetro está enviesado.

Por fim, os dados serão normalizados

In [ ]:
from sklearn import preprocessing

In [ ]:
trainX = adult_train_num[["age","education.num","capital.gain", "capital.loss", "hours.per.week","occupation",'relationship','race','sex','marital.status']]
trainY=adult_train.income
testX = adult_test_num[["age","education.num","capital.gain", "capital.loss", "hours.per.week","occupation",'relationship','race','sex','marital.status']]

In [ ]:
normalizador = preprocessing.StandardScaler()
trainX = normalizador.fit_transform(trainX)
testX = normalizador.fit_transform(testX)

Irei testar vários tipos de classificadores:

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
média=0
maior_média=0
melhor_k=0
for K in range(1,50):
    knn = KNeighborsClassifier(n_neighbors=K)
    scores = cross_val_score(knn,trainX,trainY,cv=10)
    média=scores.mean()
    if(média>maior_média):
        score_KNN=média
        melhor_k=K

In [ ]:
melhor_k

In [ ]:
knn = KNeighborsClassifier(n_neighbors=melhor_k)

In [ ]:
melhor_k

In [ ]:
score_KNN

# Naive Bayes Gaussiano

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb = GaussianNB()
scores = cross_val_score(nb,trainX,trainY,cv=10)
score_Naive_Bayes=scores.mean()

In [ ]:
score_Naive_Bayes

# Regressão Logistica

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
logreg = LogisticRegression(solver='liblinear',random_state=90)
hyperparams = dict(C=np.linspace(0,10,100), penalty=['l2', 'l1'])              
classificador_lr = RandomizedSearchCV(logreg, hyperparams, n_iter=50, cv=2, n_jobs=-1, random_state=0, verbose=2)
busca_logreg = classificador_lr.fit(trainX, trainY)
lr = LogisticRegression(solver='liblinear',random_state=90,C=busca_logreg.best_params_['C'],penalty=busca_logreg.best_params_['penalty'])
scores = cross_val_score(lr,trainX,trainY,cv=10)
score_Linear_Regression=scores.mean()
score_Regressão_Logistica=busca_logreg.best_score_
print(score_Regressão_Logistica)

Tentou-se melhorar o classificador através do AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada=AdaBoostClassifier(base_estimator=LogisticRegression(solver='liblinear',random_state=90),random_state=90)
hyperparams = dict(n_estimators=[50,100,150,200] ,learning_rate=[0.1,0.2,0.3,0.4,0.5,1,1.5])                     
classificador = RandomizedSearchCV(ada, hyperparams, n_iter=50, cv=2, n_jobs=-1, random_state=0, verbose=2)
busca = classificador.fit(trainX, trainY)
lr_boosted = AdaBoostClassifier(base_estimator=LogisticRegression(solver='liblinear',random_state=90),random_state=90,n_estimators=busca.best_params_['n_estimators'] ,learning_rate=busca.best_params_['learning_rate'])
scores = cross_val_score(lr_boosted,trainX,trainY,cv=10)
score_Linear_Regression_Boosted=scores.mean()
print(score_Linear_Regression_Boosted)

In [ ]:
busca.best_params_

# LDA e QDA

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
lda = LinearDiscriminantAnalysis()
hyperparams = dict(store_covariance=['True','False'] ,solver=['svd', 'lsqr', 'eigen'])                   
classificador_lda = RandomizedSearchCV(lda, hyperparams, n_iter=50, cv=2, n_jobs=-1, random_state=0, verbose=2)
busca_lda = classificador_lda.fit(trainX,trainY)
lda = LinearDiscriminantAnalysis(store_covariance=busca_lda.best_params_['store_covariance'] ,solver=busca_lda.best_params_['solver'])
scores = cross_val_score(lda,trainX,trainY,cv=10)
score_LDA=scores.mean()
print(score_LDA)

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
qda = QuadraticDiscriminantAnalysis()
hyperparams = dict(store_covariance=['True','False'] ,reg_param=[0,0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1])                     
classificador_qda = RandomizedSearchCV(qda, hyperparams, n_iter=50, cv=2, n_jobs=-1, random_state=0, verbose=2)
busca_qda = classificador_qda.fit(trainX, trainY)
qda = QuadraticDiscriminantAnalysis(store_covariance=busca_qda.best_params_['store_covariance'] ,reg_param=busca_qda.best_params_['reg_param'])
scores = cross_val_score(qda,trainX,trainY,cv=10)
score_QDA=scores.mean()
print(score_QDA)

# Randon Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf=RandomForestClassifier()
hyperparams = dict(criterion=['gini','entropy'] ,n_estimators=[100,150,200,250,300,350,400])                     
classificador_rf = RandomizedSearchCV(rf, hyperparams, n_iter=50, cv=2, n_jobs=-1, random_state=0, verbose=2)
busca_rf = classificador_rf.fit(trainX, trainY)
rf = RandomForestClassifier(criterion=busca_rf.best_params_['criterion'] ,n_estimators=busca_rf.best_params_['n_estimators'])
scores = cross_val_score(rf,trainX,trainY,cv=10)
score_rf=scores.mean()
print(score_rf)

# XGBOOST com árvore de classificação

In [ ]:
pip install xgboost

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(objective='binary:logistic',seed=90,subsample=0.9,colsample_bytree=0.5)
hyperparams = dict(max_depth=[1,2,3,4,5,6,7,8,9,10],learning_rate=[0.01,0.05,0.1,0.3,0.5],reg_lambda=[0,1,10],scale_pos_weight=[1,3,5,6,7])                     
classificador_xgb = RandomizedSearchCV(xgb, hyperparams, n_iter=50, cv=3, n_jobs=-1, random_state=0, verbose=2,scoring='roc_auc')
busca_xgb = classificador_xgb.fit(trainX, trainY)

In [ ]:
busca_xgb.best_params_

In [ ]:
xgb=XGBClassifier(objective='binary:logistic',seed=90,subsample=0.9,colsample_bytree=0.5,scale_pos_weight=busca_xgb.best_params_['scale_pos_weight'],reg_lambda=busca_xgb.best_params_['reg_lambda'],max_depth=busca_xgb.best_params_['max_depth'],learning_rate=busca_xgb.best_params_['learning_rate'])


In [ ]:
scores = cross_val_score(xgb,trainX,trainY,cv=10)
score_xgb=scores.mean()
score_xgb

# ADABOOST com árvore de classificação

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada=AdaBoostClassifier(random_state=90)
hyperparams = dict(n_estimators=[50,100,150,200] ,learning_rate=[0.1,0.2,0.3,0.4,0.5,1,1.5])                     
classificador = RandomizedSearchCV(ada, hyperparams, n_iter=50, cv=2, n_jobs=-1, random_state=0, verbose=2)
busca = classificador.fit(trainX, trainY)
tree_ada_boosted = AdaBoostClassifier(random_state=90,n_estimators=busca.best_params_['n_estimators'] ,learning_rate=busca.best_params_['learning_rate'])
scores = cross_val_score(tree_ada_boosted,trainX,trainY,cv=10)
score_Tree_Ada=scores.mean()
print(score_Tree_Ada)

# Submissão

In [ ]:
knn.fit(trainX,trainY)
nb.fit(trainX,trainY)
lr.fit(trainX,trainY)
lr_boosted.fit(trainX,trainY)
lda.fit(trainX,trainY)
qda.fit(trainX,trainY)
rf.fit(trainX,trainY)
xgb.fit(trainX, trainY)
tree_ada_boosted.fit(trainX,trainY)

In [ ]:
testY_knn=knn.predict(testX)
testY_nb=nb.predict(testX)
testY_lr=lr.predict(testX)
testY_lr_boosted=lr_boosted.predict(testX)
testY_lda=lda.predict(testX)
testY_qda=qda.predict(testX)
testY_rf=rf.predict(testX)
testY_xgb=xgb.predict(testX)
testY_tree_ada_boosted=tree_ada_boosted.predict(testX)

O classificador com o melhor score foi XGBOOST com árvore de classificação usando os atributos considerados, retirando mais atributos foi verificado que o score é inferior.

In [ ]:
submiss=pd.DataFrame(testY_xgb,columns=['income'])
submiss.to_csv("PMR3508-2020-90_submissao.csv", index_label="Id")

# SCORES
score_knn=0.84680

score_nb=0.81861

score_lr=0.82346

score_lr_boosted=0.82217

score_lda=0.82039

score_qda=0.81971

score_rf=0.84305

score_xgb=0.86074

score_tree_ada_boosted=0.85233

# SCORES Considerando apenas os seguintes atributos:

**Age,education.num,marital status, relationship, sex, capital gain, capital loss, hours per week**

score_knn=0.84944

score_nb=0.81443

score_lr=0.82321

score_lr_boosted=0.82248

score_lda=0.81959

score_qda=0.82076

score_rf=0.83986

score_xgb=0.75724

score_tree_ada_boosted=0.84563